In [105]:
import time
import os
import re
import glob
import struct 
import math

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.applications import vgg16

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave, imread, imresize

## Preprocessing

In [106]:
# style_image_path = 'examples/starry_johnson.jpg'
style_paths = ['examples/muse.jpg']
output_image_path = 'muse_out/'

content_weight = 0.025
style_weights = [1]
temporal_weight = 0.05
total_variation_weight = 5e-8

img_nrows = 540
img_ncols = 960

# Utility function to convert a tensor into a valid image
def deprocess_image(x):
    x = x.reshape((3, img_nrows, img_ncols))
    x = x.transpose((1, 2, 0))

    x = x[:, :, ::-1]
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def preprocess_image(image_path):
    img = imread(image_path)  # Prevents crashes due to PNG images (ARGB)

    img = imresize(img, (img_nrows, img_ncols)).astype('float32')

    # RGB -> BGR
    img = img[:, :, ::-1]

    img[:, :, 0] -= 103.939
    img[:, :, 1] -= 116.779
    img[:, :, 2] -= 123.68

    if K.image_dim_ordering() == "th":
        img = img.transpose((2, 0, 1)).astype('float32')

    img = np.expand_dims(img, axis=0)
    return img

## Define Loss Functions

In [107]:
# The gram matrix of an image tensor (feature-wise outer product)
def gram_matrix(x):
    assert K.ndim(x) == 3
    features = K.batch_flatten(x)
    gram = K.dot(features, K.transpose(features))
    return gram

# The "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image and from the generated image
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

# An auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# An auxiliary loss function
# designed to maintain the temporal consistency
def temporal_loss(warped, combination, weights):
    return K.sum(weights * K.square(combination - warped))

# The 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(x[:, :, :img_nrows-1, :img_ncols-1] - x[:, :, 1:, :img_ncols-1])
    b = K.square(x[:, :, :img_nrows-1, :img_ncols-1] - x[:, :, :img_nrows-1, 1:])

    return K.sum(K.pow(a + b, 1.25))

## Style transfer with temporal constraints
## Add titu1994 improvements on the keras neural transfer example

In [119]:
def static_transfer(content_image_path, n_frame, x, weights):
    content_image = K.variable(preprocess_image(content_image_path))
    input_arr = [content_image]
    
    for style_image_path in style_paths:
        input_arr.append(K.variable(preprocess_image(style_image_path)))

    # This will contain our generated image
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
    input_arr.append(combination_image)

    input_tensor_content = K.concatenate(input_arr, axis=0)
    n_tensors = len(input_arr)

    model = vgg16.VGG16(input_tensor=input_tensor_content, weights='imagenet',
                  include_top=False)
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
    

    loss = K.variable(0.)
    layer_features = outputs_dict['block5_conv2']
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[n_tensors - 1, :, :, :]
    cls = content_weight * content_loss(base_image_features, combination_features)
    loss += cls

    # Style_Loss
    feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

    feature_layers += ['block1_conv2', 'block2_conv2',
                      'block3_conv3', 'block4_conv3',
                      'block5_conv3']
    n_layers = len(feature_layers) - 1
    n_styles = len(style_paths)

    sls = K.variable(0.)       

    # Improvement 3 : Chained Inference without blurring
    for i in range(n_layers):
        layer_features_1 = outputs_dict[feature_layers[i]]
        combination_features = layer_features_1[n_tensors - 1, :, :, :]
                
        sl_1 = []
        for j in range(n_styles):            
            style_reference_features = layer_features_1[j + 1, :, :, :]        
            sl_1.append(style_loss(style_reference_features, combination_features))
                    
        layer_features_2 = outputs_dict[feature_layers[i + 1]]
        combination_features = layer_features_2[n_tensors - 1, :, :, :]
    
        sl_2 = []
        for j in range(n_styles):            
            style_reference_features = layer_features_2[j+1, :, :, :]        
            sl_2.append(style_loss(style_reference_features, combination_features))    
        
        for j in range(n_styles):
            sl = sl_1[j] - sl_2[j]

            # Improvement 4
            # Geometric weighted scaling of style loss
            loss += (style_weights[j] / (2 ** (n_layers - (i + 1)))) * sl
            sls += (style_weights[j] / (2 ** (n_layers - (i + 1)))) * sl    
            
    # Temporal Loss
    tls = K.variable(0.)  
        
    input_layer_name = [layer.name for layer in model.layers if 'input' in layer.name][0]
    layer_features = outputs_dict[input_layer_name]
    combination_features = layer_features[n_tensors - 1, :, :, :]
    
    if None != weights:
        warped = x.copy()
        weights = K.variable(np.expand_dims(weights, axis=0)/255)
        tl = temporal_loss(K.variable(warped), combination_features, weights)
        tls = temporal_weight * tl
        loss += temporal_weight * tl
    else:
        tls += 0        
    
    loss += total_variation_weight * total_variation_loss(combination_image)
    tvs = total_variation_weight * total_variation_loss(combination_image)
    
    # Get the gradients of the generated image wrt the loss
    grads = K.gradients(loss, combination_image)

    outputs = [loss]
    outputs.append(cls)
    outputs.append(sls)
    outputs.append(tls)
    outputs.append(tvs)
    
    if type(grads) in {list, tuple}:
        outputs += grads
    else:
        outputs.append(grads)

    f_outputs = K.function([combination_image], outputs)
    
    def eval_loss_and_grads(x):
        x = x.reshape((1, 3, img_nrows, img_ncols))

        outs = f_outputs([x])
        loss_value = outs[0]                
#         print('current', [int(x) for x in outs[:5]])
        
        if len(outs[5:]) == 1:
            grad_values = outs[5].flatten().astype('float64')
        else:
            grad_values = np.array(outs[5:]).flatten().astype('float64')
        return loss_value, grad_values
    
    class Evaluator(object):
        def __init__(self):
            self.loss_value = None
            self.grads_values = None

        def loss(self, x):
            assert self.loss_value is None
            loss_value, grad_values = eval_loss_and_grads(x)
            self.loss_value = loss_value
            self.grad_values = grad_values
            return self.loss_value

        def grads(self, x):
            assert self.loss_value is not None
            grad_values = np.copy(self.grad_values)
            self.loss_value = None
            self.grad_values = None
            return grad_values

    
    evaluator = Evaluator()
    iterations = 30 if n_frame==1 else 8
    # Run scipy-based optimization (L-BFGS) over the pixels of the generated image
    # so as to minimize the neural style loss


    for i in range(iterations):
        print('Start of iteration', i)
        start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                         fprime=evaluator.grads, maxfun=20)
        print('Current loss value:', min_val)
        # save current generated image
        if i == iterations - 1:
            img = deprocess_image(x.copy())
            fname = os.path.join(output_image_path,
                         'static_%04d.png' % n_frame)
            imsave(fname, img)
            print('Image saved as', fname)
        end_time = time.time()

        print('Iteration %d completed in %ds' % (i, end_time - start_time))
    return x

## Read flow files and warp image.

In [110]:
# read binary optical flow file
def read_flows(filename):
    with open(filename, mode='rb') as file: # b is important -> binary
        file_content = file.read()

    width, height = struct.unpack("ii", file_content[4:12])
    flows_content = file_content[12:]
    flows_data = struct.unpack("f" * (len(flows_content) // 4), flows_content)

    flows = np.zeros((2, height, width))
    for y in range(height):
        for x in range(width):
            shift = y*width + x
            flows[1][y][x] = flows_data[2*shift]
            flows[0][y][x] = flows_data[2*shift+1]
    return flows

In [111]:
# Warp the image with optical flow
def bilinear_interpolate(src, flows):
    dst = np.zeros_like(src)

    src_height = src.shape[1]
    src_width = src.shape[2]

    height = flows.shape[1]
    width = flows.shape[2]

    assert height == src_height
    assert width == src_width

    channels = 3
    pad_value = 0

    for y in range(height):
        for x in range(width):
            flow_y = flows[0][y][x]
            flow_x = flows[1][y][x]
            iy = y + flow_y
            ix = x + flow_x

            off_image = 0
            if iy<0 or iy>src_height-1 or ix<0 or ix>src_width-1:
                off_image = 1

            if off_image==1:
                for k in range(channels):
                    dst[k][y][x] = pad_value
            else:                
                ix_nw = math.floor(ix)
                iy_nw = math.floor(iy);
                ix_ne = ix_nw + 1;
                iy_ne = iy_nw;
                ix_sw = ix_nw;
                iy_sw = iy_nw + 1;
                ix_se = ix_nw + 1;
                iy_se = iy_nw + 1;

                # get surfaces to each neighbor:
                nw = (ix_se - ix)*(iy_se - iy);
                ne = (ix - ix_sw)*(iy_sw - iy);
                sw = (ix_ne - ix)*(iy - iy_ne);
                se = (ix - ix_nw)*(iy - iy_nw);    

                # weighted sum of neighbors:
                for k in range(channels):
                    dst[k][y][x] = src[k][iy_nw][ix_nw] * nw \
                                 + src[k][iy_ne][min(ix_ne, src_width-1)] * ne \
                                 + src[k][min(iy_sw, src_height-1)][ix_sw] * sw \
                                 + src[k][min(iy_se, src_height-1)][min(ix_se, src_width-1)] * se
    return dst
                    

## Process All Frames

In [112]:
frames = [path for path in glob.glob('muse/frame*.ppm')]
frames = sorted([(int(re.findall(r'00(\d+)', frame)[0]), frame) for frame in frames])

In [1]:
content_image_path = 'muse/frame_0001.ppm'
prev = preprocess_image(content_image_path)
for frame in frames[1:]:
    print('processing %s' % frame[1])
    if frame[0] > 11:
        flowfile = 'dragon_flows/backward_{:d}_{:d}.flo'.format(frame[0], frame[0] - 1)
        flows = read_flows(flowfile)
        
        weightfile = 'dragon_flows/reliable_{:d}_{:d}.pgm'.format(frame[0], frame[0] - 1)
        weights = img_to_array(load_img(weightfile))
 
        prev_path = 'muse_out/static_{:04d}.png'.format(frame[0] - 1)     
        prev = preprocess_image(prev_path)[0]        
        prev_warped = np.expand_dims(bilinear_interpolate(prev, flows), axis=0)    
    
        _ = static_transfer(frame[1], frame[0], prev_warped, weights)   
    else:
        _ = static_transfer(frame[1], frame[0], prev, None) 
    
    